# BIC - CW

### Importing the libraries

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf

import pyswarms as ps
from pyswarms.utils.functions import single_obj as fx
from pyswarms.utils.functions.single_obj import sphere as f

import pyswarms.backend as P
from pyswarms.backend.topology import Star

%load_ext autoreload
%autoreload 2

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [3]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [4]:
print(y)

[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [6]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN and adding custom Gaussian, Cosine and Null Activation Functions

In [11]:
ann = tf.keras.models.Sequential()
K = tf.keras.backend
utils = tf.keras.utils
def gaussian(x):
    return K.exp(-K.pow(x,2)/2)
def cosine(x) :
    return K.cos(x)
def nullact(x) :
    return K.constant(0)*x

utils.get_custom_objects().update({'gaussian': tf.keras.layers.Activation(gaussian)})
utils.get_custom_objects().update({'cosine': tf.keras.layers.Activation(cosine)})
utils.get_custom_objects().update({'nullact': tf.keras.layers.Activation(nullact)})

### Number of Neurons and Layers Configuration

In [12]:
neurons = int(input("Enter number of neurons: "))

Enter number of neurons: 6


In [13]:
layers = []
n = int(input("Enter number of layers (1-5): "))
if n < 6 and n > 0:
    for i in range (0, n):
        print("Hyperbolic tangent layer must run every time.")
        print("Available activation functions: linear, sigmoid, gaussian, cosine, null")
        layer = input("Enter name of desired activation function: ")
    
        layers.append(layer)    
else:
    print("Wrong number of layers entered")

Enter number of layers (1-5): 4
Hyperbolic tangent layer must run every time.
Available activation functions: sigmoid, gaussian, cosine, null
Enter name of desired activation function: linear
Hyperbolic tangent layer must run every time.
Available activation functions: sigmoid, gaussian, cosine, null
Enter name of desired activation function: sigmoid
Hyperbolic tangent layer must run every time.
Available activation functions: sigmoid, gaussian, cosine, null
Enter name of desired activation function: gaussian
Hyperbolic tangent layer must run every time.
Available activation functions: sigmoid, gaussian, cosine, null
Enter name of desired activation function: cosine


### Adding the input layer and the first hidden layer with linear AF

In [14]:
if "linear" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='relu'))

### Adding the second hidden layer with Sigmoid AF

In [15]:
if "sigmoid" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='sigmoid'))

### Adding the third hidden layer with Gaussian AF

In [16]:
if "gaussian" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='gaussian'))

### Adding the fourth hidden layer with Cosine AF

In [17]:
if "cosine" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='cosine'))

### Adding the fifth hidden layer with Null AF

In [18]:
if "null" in layers:
    ann.add(tf.keras.layers.Dense(units=neurons, activation='nullact'))

### Adding the output layer with Hyperbolic Tangent AF

In [19]:
ann.add(tf.keras.layers.Dense(units=1, activation='tanh'))

## Part 3 - Training the ANN

### Compiling the ANN

In [20]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [54]:
def forward_prop(params):
    
    for x_train, y_train in dataset:
        logits = ann(params)
    
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Compute for the negative log likelihood

    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

    return loss

In [55]:
def f(x):
    n_particles = x.shape[0]
    print(n_particles)
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [56]:
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = 54
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)

2020-11-20 21:39:20,366 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/1000

100


ValueError: too many values to unpack (expected 2)

In [21]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 485us/step - loss: 0.9287 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 447us/step - loss: 0.5014 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 465us/step - loss: 0.4822 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 450us/step - loss: 0.4649 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 553us/step - loss: 0.4584 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 456us/step - loss: 0.4548 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 479us/step - loss: 0.4798 - accuracy: 0.7884
Epoch 8/100
250/250 [==============================] - 0s 449us/step - loss: 0.4519 - accuracy: 0.7966
Epoch 9/100
250/250 [==============================] - 0s 483us/step - loss: 0.4538 - accuracy: 0.7958
Epoch 10/100
250/250 [==============================] - 0s 444us/step - l

250/250 [==============================] - 0s 571us/step - loss: 0.3363 - accuracy: 0.8643
Epoch 80/100
250/250 [==============================] - 0s 505us/step - loss: 0.3361 - accuracy: 0.8656
Epoch 81/100
250/250 [==============================] - 0s 455us/step - loss: 0.3352 - accuracy: 0.8646
Epoch 82/100
250/250 [==============================] - 0s 456us/step - loss: 0.3379 - accuracy: 0.8665
Epoch 83/100
250/250 [==============================] - 0s 468us/step - loss: 0.3355 - accuracy: 0.8640
Epoch 84/100
250/250 [==============================] - 0s 476us/step - loss: 0.3361 - accuracy: 0.8659
Epoch 85/100
250/250 [==============================] - 0s 485us/step - loss: 0.3440 - accuracy: 0.8665
Epoch 86/100
250/250 [==============================] - 0s 458us/step - loss: 0.3341 - accuracy: 0.8666
Epoch 87/100
250/250 [==============================] - 0s 515us/step - loss: 0.3357 - accuracy: 0.8649
Epoch 88/100
250/250 [==============================] - 0s 536us/step - loss:

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Solution**

In [22]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


### Predicting the Test set results

In [23]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1513   82]
 [ 194  211]]


0.862